In [1]:
startSwank 4011

Checking SLIME_HOME
About to start swank server
Started swank server


T


;; Swank started at port: 4011.

;; swank:close-connection: Unexpected end of file on #<TWO-WAY-STREAM >.


In [ ]:
(setf fep::*testing* nil)

In [ ]:
(setf *default-pathname-defaults* #P"~/Development/fep-benchmark/")

In [ ]:
(asdf:load-asd "/Users/Yonkunas/Development/lomap/lomap.asd")

In [ ]:
(asdf:load-system :lomap)

In [ ]:
(setup-default-paths)

In [ ]:
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

In [ ]:
(source "/usr/local/amber18/dat/leap/cmd/oldff/leaprc.ff14SB.redq")

In [ ]:
(source "leaprc.gaff")

In [ ]:
(leap:add-pdb-res-map '((1 :NMA :NME)))

In [ ]:
(leap:add-pdb-atom-map '((:CA :CH3) (:1HA :HH31) (:2HA :HH32) (:3HA :HH33)
                         (:1H :HH31) (:2H :HH32) (:3H :HH33)
                         (:H2 :H)))

In [ ]:
(defparameter *tg* (molecule-graph.max-clique::test-cross-product))

In [ ]:
(molecule-graph.max-clique::cross-product-edges *tg*)

In [ ]:
(defparameter *td* (molecule-graph.max-clique::build-dimacs *tg*))

In [ ]:
(defparameter *mc* (molecule-graph.max-clique::dothing *td*))

In [ ]:
*mc*

In [ ]:
(molecule-graph.max-clique::extract-atom-pairs *tg* *mc*)

In [ ]:
(defparameter mols (sdf:load-sdf-as-list-of-molecules "eg5/ligands_original.sdf"))

In [ ]:
(lomap::similarity-matrix mols)

In [ ]:
(class-of mols)

In [ ]:
(length mols)

In [ ]:
(molecule-graph.max-clique::rename-atoms (first mols) "a")
(molecule-graph.max-clique::rename-atoms (second mols) "b")

In [ ]:
(defparameter *cg* (multiple-value-list (molecule-graph.max-clique::compare-molecules (first mols) (second mols))))

In [ ]:
(lomap::mcsr-similarity-score (first mols) (second mols))

In [ ]:
*cg*

In [ ]:
(length (first *cg*))

In [ ]:
(+ 19 2)

In [ ]:
(+ 19 4)

In [ ]:
(- (+ 23 21) (* 2 19))

In [ ]:
*cg*

In [ ]:
(multiple-value-list (molecule-graph:compare-molecules (first mols) (second mols) 
                                                       :atom-match-callback 'molecule-graph:element-match))

In [ ]:
firstsk = (sketch2d:sketch2d (first mols))

In [ ]:
nthsk = (sketch2d:sketch2d (nth 20 mols))

In [ ]:
(first mols)

In [ ]:
(show nthsk)

In [ ]:
(show firstsk)

In [ ]:
(let ((sketch2d::*show-names* t)) (show (sketch2d:sketch2d (first mols))))

In [ ]:
show (defparameter *sk* (sketch2d:similar-sketch2d (nth 18 mols) firstsk))

In [ ]:
show (defparameter *sk* (sketch2d:similar-sketch2d (third mols) firstsk :accumulate-coordinates t))

In [ ]:
(length mols)

The following compares every molecule to every other molecule - it will take a LOOONG time

In [ ]:
(loop for xi below (1- (length mols))
      for molx = (elt mols xi)
      do (format t "xi -> ~a~%" xi)
      do (finish-output)
      do (loop for yi from (1+ xi) below (length mols)
               for moly = (elt mols yi)
               do (format t "   yi -> ~a~%" yi)
               do (finish-output)
               do (molecule-graph.max-clique:compare-molecules molx moly)))

In [ ]:
(defparameter *d* (sketch2d::dynamics *sk*))

In [ ]:
show *d*

In [ ]:
(defparameter feps (fep:fep-calculation-from-ligands mols))

In [ ]:
(defparameter receptor (load-pdb "eg5/3l9h_prepared.pdb"))

In [ ]:
;; For some reason the waters in receptor have residue name 'cando-user::tp3
;;  and they need to be :wat
(let ((changed 0))
  (cando:do-residues (res receptor)
    (when (eq (chem:get-name res) 'cando-user::tp3)
      (incf changed)
      (chem:set-name res :wat)))
  (format t "Changed ~d residue names from 'cando-user::tp3 to :wat~%" changed))

In [ ]:
(push receptor (fep:receptors feps))

In [ ]:
(fep:build-job-nodes feps)

In [ ]:
(fep:connect-job-nodes feps '((:chembl1082248 :chembl1083517)
                              (:chembl1082248 :chembl1084143)
                              #+(or)(:chembl1088740 :chembl1093088)))

In [ ]:
(fep:check-calculation-atom-types feps)

In [ ]:
(let* ((*default-pathname-defaults* (merge-pathnames #P"cando-fep-benchmarks/eg5/full/"))
       (worklist
         (progn
           (format t "generate-jobs to *default-pathname-defaults* -> ~s~%" *default-pathname-defaults*)
           (fep:generate-jobs feps))))
  (with-open-file (fout #P"jobs/conf.sh" :direction :output :if-exists :supersede)
    (format fout "pmemd_cuda=pmemd.cuda
execute_cpu_local=0
execute_gpu_local=0
execute_lisp_local=0
worker=schando
distributor=s103.thirdlaw.tech
lisp_jobs_only_on=172.234.2.1
"))
  (with-open-file (sout "/tmp/eg5.dot" :direction :output)
    (fepdot:draw-graph-stream (list worklist) sout))
  )